### Checking for duplicates in the two main columns of Standardized hu.MAP 2.0 CSV

I observed some duplicates in the `Uniprot_ACCs` & `genenames` columns in the **raw** hu.MAP 2.0 data (see `Check_for_duplicates_in_author_provided_hu.MAP2.0_csv.ipynb`) and I took steps to address that, see `Standardizing_identifier_order_in_humap2-provided_csv.ipynb`.)  
This checks if successfully handled, compared to raw.

------

#### Preparation

##### Get the complexes with confidence scores

Get the fixed standardized CSV:

In [4]:
!curl -OL https://raw.githubusercontent.com/fomightez/humap2-binder/refs/heads/main/additional_nbs/standardizing_initial_data/humap2_complexes_20200809InOrderMatched.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1243k  100 1243k    0     0  2149k      0 --:--:-- --:--:-- --:--:-- 2152k


##### Put the data on the complexes into Pandas dataframe

(I'm using uv here just because I want to learn about it. I could have run the code in the script right in this notebook, and skipped the pickling and read pickle steps.)

Get the script to use with `uv` to read in the raw data and make a dataframe.

In [5]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/complexes_rawCSV_to_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1007  100  1007    0     0   3460      0 --:--:-- --:--:-- --:--:--  3460


In [1]:
!uv run complexes_rawCSV_to_df.py humap2_complexes_20200809InOrderMatched.csv
import pandas as pd
rd_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
rd_df

Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               

,HuMAP2_ID,Confidence,Uniprot_ACCs,genenames
0,HuMAP2_00000,3,Q9BQS8 O95900,FYCO1 TRUB2
1,HuMAP2_00001,4,Q15102 P68402 Q15797 P08133 Q99426 Q9H4M9,PAFAH1B3 PAFAH1B2 SMAD1 ANXA6 TBCB EHD1
2,HuMAP2_00002,5,A1KXE4 Q9UF11 Q6ZRY4 Q15038 O43251 Q9Y6M7 Q154...,FAM168B PLEKHB1 RBPMS2 DAZAP2 RBFOX2 SLC4A7 RB...
3,HuMAP2_00003,5,O14974 Q8WUM9 Q9Y5Y0 Q16563 Q15836 Q14919 Q9BW...,PPP1R12A SLC20A1 FLVCR1 SYPL1 VAMP3 DRAP1 PHF7...
4,HuMAP2_00004,4,Q49A92 Q8WV99 Q9NQT8 Q9H672 P20774,C8orf34 ZFAND2B KIF13B ASB7 OGN
...,...,...,...,...
6960,HuMAP2_07014,4,Q9HC97 Q6S8J3 P31152 P13727 Q92871,GPR35 POTEE MAPK4 PRG2 PMM1
6961,HuMAP2_07015,4,Q8N5N7 Q96E29 O75127 Q96I51 Q9H5L6 Q9NPE2,MRPL50 MTERF3 PTCD1 RCC1L THAP9 NGRN
6962,HuMAP2_07016,5,Q9UPQ7 Q2T9J0 Q8NE31 O75319 P52952 Q96HN2 O438...,PDZRN3 TYSND1 FAM13C DUSP11 NKX2-5 AHCYL2 AHCY...
6963,HuMAP2_07017,2,Q96GP6 Q53GT1 P49448,SCARF2 KLHL22 GLUD2


That's a lot of complexes!

--------

### Now to check for duplicates

In [2]:
# check for duplicates in the two main columns
rows_with_issue_content = []
for row in rd_df.itertuples():
    accs = row.Uniprot_ACCs.split()
    set_of_accs = set(accs)
    if len(accs) != len(set_of_accs):
        print(f"`{row.Uniprot_ACCs}` displays duplicates.")
        rows_with_issue_content.append(row)
    gns = row.genenames.split()
    # remove those with `SPECIALin_UniProt_but_no_gene` from consideration because can occur more than once to balance
    identifiers_not_to_consider = ['SPECIALin_UniProt_but_no_gene']
    gns = [x for x in gns if x not in identifiers_not_to_consider]
    set_of_gns = set(gns)
    if len(gns) != len(set_of_gns):
        print(f"`{row.genenames}` displays duplicates.")
        rows_with_issue_content.append(row)
if not rows_with_issue_content:
    import rich
    rich.print("[bold]There are no occurences of duplicate identifiers observed in the two main columns[/bold].")

There are no occurences of duplicate identifiers observed in the two main columns.

-----

Enjoy!